# Train Poisson PINN - Day 4 Task 2

This notebook trains a PINN to solve the 2D Poisson equation and achieve <1% relative L2 error.

**Configuration:**
- Model: 4 hidden layers, 64 neurons each, tanh activation
- Optimizer: Adam (lr=1e-3)
- Training: 20,000 epochs
- Collocation points: 10,000 interior + 400 boundary
- Target: Relative L2 error < 1%

**Instructions:**
1. Enable GPU: Runtime → Change runtime type → GPU
2. Run all cells
3. Download the trained model and results
4. Continue with Task 3 locally

## Setup

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import matplotlib.pyplot as plt
from typing import Dict, Optional, Tuple, Callable
import time
import os

# Check GPU availability
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")

## Define MLP Model

In [ ]:
class MLP(nn.Module):
    """Multi-Layer Perceptron for PINN."""
    
    def __init__(self, input_dim: int, hidden_dims: list, output_dim: int, activation: str = "tanh"):
        super().__init__()
        self.input_dim = input_dim
        self.hidden_dims = hidden_dims
        self.output_dim = output_dim
        self.activation_name = activation
        
        # Build layers
        dims = [input_dim] + hidden_dims + [output_dim]
        self.layers = nn.ModuleList([
            nn.Linear(dims[i], dims[i+1]) for i in range(len(dims)-1)
        ])
        
        # Activation function
        if activation == "tanh":
            self.activation_fn = torch.tanh
        elif activation == "relu":
            self.activation_fn = torch.relu
        elif activation == "gelu":
            self.activation_fn = torch.nn.functional.gelu
        elif activation == "sin":
            self.activation_fn = torch.sin
        else:
            raise ValueError(f"Unknown activation: {activation}")
        
        # Xavier initialization
        self._initialize_weights()
    
    def _initialize_weights(self):
        for layer in self.layers:
            nn.init.xavier_normal_(layer.weight)
            nn.init.zeros_(layer.bias)
    
    def forward(self, x: torch.Tensor) -> torch.Tensor:
        h = x
        for layer in self.layers[:-1]:
            h = layer(h)
            h = self.activation_fn(h)
        h = self.layers[-1](h)
        return h

## Define Poisson Problem

In [ ]:
class PoissonProblem:
    """2D Poisson equation: ∇²u = f on [0,1]²."""
    
    def __init__(self):
        self.spatial_dim = 2
        self.domain = [[0.0, 1.0], [0.0, 1.0]]
    
    def analytical_solution(self, x: torch.Tensor) -> torch.Tensor:
        """u(x,y) = sin(πx)sin(πy)"""
        return torch.sin(np.pi * x[:, 0:1]) * torch.sin(np.pi * x[:, 1:2])
    
    def source_term(self, x: torch.Tensor) -> torch.Tensor:
        """f(x,y) = -2π²sin(πx)sin(πy)"""
        return -2 * np.pi**2 * torch.sin(np.pi * x[:, 0:1]) * torch.sin(np.pi * x[:, 1:2])
    
    def boundary_condition(self, x: torch.Tensor) -> torch.Tensor:
        """Dirichlet BC: u = 0 on ∂Ω"""
        return torch.zeros((x.shape[0], 1), device=x.device)
    
    def pde_residual(self, u: torch.Tensor, x: torch.Tensor, 
                     du_dx: torch.Tensor, d2u_dx2: torch.Tensor) -> torch.Tensor:
        """PDE residual: ∇²u - f"""
        laplacian = d2u_dx2[:, 0:1] + d2u_dx2[:, 1:2]
        source = self.source_term(x)
        return laplacian - source
    
    def sample_interior_points(self, n: int, random_seed: Optional[int] = None) -> torch.Tensor:
        """Sample interior points using Latin Hypercube."""
        if random_seed is not None:
            np.random.seed(random_seed)
        
        from scipy.stats import qmc
        sampler = qmc.LatinHypercube(d=2, seed=random_seed)
        samples = sampler.random(n=n)
        
        # Scale to domain
        x = samples[:, 0] * (self.domain[0][1] - self.domain[0][0]) + self.domain[0][0]
        y = samples[:, 1] * (self.domain[1][1] - self.domain[1][0]) + self.domain[1][0]
        
        return torch.tensor(np.column_stack([x, y]), dtype=torch.float32)
    
    def sample_boundary_points(self, n_per_edge: int, 
                               random_seed: Optional[int] = None) -> torch.Tensor:
        """Sample boundary points uniformly on edges."""
        x_min, x_max = self.domain[0]
        y_min, y_max = self.domain[1]
        
        x = torch.linspace(x_min, x_max, n_per_edge)
        y = torch.linspace(y_min, y_max, n_per_edge)
        
        # Bottom, top, left, right edges
        bottom = torch.stack([x, torch.full_like(x, y_min)], dim=1)
        top = torch.stack([x, torch.full_like(x, y_max)], dim=1)
        left = torch.stack([torch.full_like(y, x_min), y], dim=1)
        right = torch.stack([torch.full_like(y, x_max), y], dim=1)
        
        return torch.cat([bottom, top, left, right], dim=0)
    
    def compute_relative_l2_error(self, model: nn.Module, n_test_points: int = 10000,
                                   random_seed: Optional[int] = None) -> float:
        """Compute relative L2 error."""
        model.eval()
        x_test = self.sample_interior_points(n_test_points, random_seed)
        x_test = x_test.to(next(model.parameters()).device)
        
        with torch.no_grad():
            u_pred = model(x_test)
            u_exact = self.analytical_solution(x_test)
            
            error = torch.mean((u_pred - u_exact) ** 2)
            norm = torch.mean(u_exact ** 2)
            relative_error = torch.sqrt(error / norm) * 100
        
        return relative_error.item()

## Derivative Utilities

In [ ]:
def compute_derivatives(u: torch.Tensor, x: torch.Tensor, order: int = 1) -> torch.Tensor:
    """Compute derivatives using automatic differentiation."""
    if order == 1:
        # First-order derivatives
        grads = []
        for i in range(x.shape[1]):
            grad = torch.autograd.grad(
                outputs=u,
                inputs=x,
                grad_outputs=torch.ones_like(u),
                create_graph=True,
                retain_graph=True,
            )[0][:, i:i+1]
            grads.append(grad)
        return torch.cat(grads, dim=1)
    
    elif order == 2:
        # Second-order derivatives (diagonal of Hessian)
        du_dx = compute_derivatives(u, x, order=1)
        hessian_diag = []
        for i in range(x.shape[1]):
            d2u_dx2 = torch.autograd.grad(
                outputs=du_dx[:, i:i+1],
                inputs=x,
                grad_outputs=torch.ones_like(du_dx[:, i:i+1]),
                create_graph=True,
                retain_graph=True,
            )[0][:, i:i+1]
            hessian_diag.append(d2u_dx2)
        return torch.cat(hessian_diag, dim=1)
    
    else:
        raise ValueError(f"Order {order} not supported")

## Training Loop

In [ ]:
def train_pinn(model, problem, device, n_epochs=20000, n_interior=10000, n_boundary=100,
               lr=1e-3, validate_every=100, print_every=500):
    """Train PINN on Poisson equation."""
    
    model = model.to(device)
    optimizer = optim.Adam(model.parameters(), lr=lr)
    
    history = {
        'loss_total': [],
        'loss_pde': [],
        'loss_bc': [],
        'relative_l2_error': [],
    }
    
    print("="*80)
    print("Starting PINN Training")
    print("="*80)
    print(f"Device: {device}")
    print(f"Interior points: {n_interior}")
    print(f"Boundary points: {n_boundary * 4}")
    print(f"Epochs: {n_epochs}")
    print("="*80)
    print()
    
    start_time = time.time()
    
    for epoch in range(1, n_epochs + 1):
        model.train()
        
        # Sample collocation points
        x_interior = problem.sample_interior_points(n_interior).to(device)
        x_boundary = problem.sample_boundary_points(n_boundary).to(device)
        
        # Enable gradients for PDE residual
        x_interior = x_interior.requires_grad_(True)
        
        # Forward pass - interior
        u_interior = model(x_interior)
        du_dx = compute_derivatives(u_interior, x_interior, order=1)
        d2u_dx2 = compute_derivatives(u_interior, x_interior, order=2)
        residual = problem.pde_residual(u_interior, x_interior, du_dx, d2u_dx2)
        loss_pde = torch.mean(residual ** 2)
        
        # Forward pass - boundary
        u_boundary = model(x_boundary)
        bc_exact = problem.boundary_condition(x_boundary)
        loss_bc = torch.mean((u_boundary - bc_exact) ** 2)
        
        # Total loss
        loss_total = loss_pde + loss_bc
        
        # Backward pass
        optimizer.zero_grad()
        loss_total.backward()
        optimizer.step()
        
        # Log history
        history['loss_total'].append(loss_total.item())
        history['loss_pde'].append(loss_pde.item())
        history['loss_bc'].append(loss_bc.item())
        
        # Validation
        if epoch % validate_every == 0:
            rel_error = problem.compute_relative_l2_error(model, n_test_points=5000)
            history['relative_l2_error'].append(rel_error)
        else:
            rel_error = None
        
        # Print progress
        if epoch % print_every == 0:
            elapsed = time.time() - start_time
            print(f"Epoch {epoch:5d}/{n_epochs} | "
                  f"Loss: {loss_total.item():.6f} "
                  f"(PDE: {loss_pde.item():.6f}, BC: {loss_bc.item():.6f}) | "
                  f"Time: {elapsed:.1f}s", end="")
            if rel_error is not None:
                print(f" | L2 Error: {rel_error:.4f}%")
            else:
                print()
    
    # Final validation
    final_error = problem.compute_relative_l2_error(model, n_test_points=10000)
    history['relative_l2_error'].append(final_error)
    
    elapsed = time.time() - start_time
    print()
    print("="*80)
    print("Training Complete!")
    print("="*80)
    print(f"Total time: {elapsed:.2f}s ({elapsed/60:.2f} minutes)")
    print(f"Final loss: {history['loss_total'][-1]:.6f}")
    print(f"Final relative L2 error: {final_error:.4f}%")
    if final_error < 1.0:
        print("✅ SUCCESS: Achieved target error < 1%")
    else:
        print(f"⚠️  Target not reached. Current: {final_error:.4f}%")
    print("="*80)
    
    return model, history

## Run Training

In [ ]:
# Create model and problem
model = MLP(input_dim=2, hidden_dims=[64, 64, 64, 64], output_dim=1, activation="tanh")
problem = PoissonProblem()

print(f"Model parameters: {sum(p.numel() for p in model.parameters()):,}")
print(f"Device: {device}")
print()

# Train
trained_model, history = train_pinn(
    model=model,
    problem=problem,
    device=device,
    n_epochs=20000,
    n_interior=10000,
    n_boundary=100,
    lr=1e-3,
    validate_every=100,
    print_every=500,
)

## Visualize Results

In [ ]:
# Plot training history
fig, axes = plt.subplots(1, 2, figsize=(12, 4))

# Loss curves
axes[0].plot(history['loss_total'], label='Total Loss', linewidth=2)
axes[0].plot(history['loss_pde'], label='PDE Loss', alpha=0.7)
axes[0].plot(history['loss_bc'], label='BC Loss', alpha=0.7)
axes[0].set_xlabel('Epoch')
axes[0].set_ylabel('Loss')
axes[0].set_yscale('log')
axes[0].set_title('Training Loss Curves')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# Relative L2 error
validation_epochs = [i * 100 for i in range(len(history['relative_l2_error']))]
axes[1].plot(validation_epochs, history['relative_l2_error'], marker='o', linewidth=2, markersize=3)
axes[1].set_xlabel('Epoch')
axes[1].set_ylabel('Relative L2 Error (%)')
axes[1].set_title('Validation Error')
axes[1].axhline(y=1.0, color='r', linestyle='--', alpha=0.5, label='1% target')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('training_history.png', dpi=150, bbox_inches='tight')
plt.show()

print("Training history saved to: training_history.png")

In [ ]:
# Generate solution heatmap
model.eval()
n_points = 100

x = torch.linspace(0, 1, n_points)
y = torch.linspace(0, 1, n_points)
X, Y = torch.meshgrid(x, y, indexing='ij')
grid_points = torch.stack([X.flatten(), Y.flatten()], dim=1).to(device)

with torch.no_grad():
    u_pred = model(grid_points).cpu().numpy().reshape(n_points, n_points)
    u_exact = problem.analytical_solution(grid_points.cpu()).numpy().reshape(n_points, n_points)

error = np.abs(u_pred - u_exact)

fig, axes = plt.subplots(1, 3, figsize=(15, 5))

X_np = X.numpy()
Y_np = Y.numpy()

# PINN Solution
im1 = axes[0].contourf(X_np, Y_np, u_pred, levels=50, cmap='viridis')
axes[0].set_title('PINN Solution', fontsize=12, fontweight='bold')
axes[0].set_xlabel('x')
axes[0].set_ylabel('y')
axes[0].set_aspect('equal')
plt.colorbar(im1, ax=axes[0], label='u(x, y)')

# Analytical Solution
im2 = axes[1].contourf(X_np, Y_np, u_exact, levels=50, cmap='viridis')
axes[1].set_title('Analytical Solution', fontsize=12, fontweight='bold')
axes[1].set_xlabel('x')
axes[1].set_ylabel('y')
axes[1].set_aspect('equal')
plt.colorbar(im2, ax=axes[1], label='u(x, y)')

# Absolute Error
im3 = axes[2].contourf(X_np, Y_np, error, levels=50, cmap='hot')
axes[2].set_title(f'Absolute Error\n(Max: {error.max():.2e}, Mean: {error.mean():.2e})',
                  fontsize=12, fontweight='bold')
axes[2].set_xlabel('x')
axes[2].set_ylabel('y')
axes[2].set_aspect('equal')
plt.colorbar(im3, ax=axes[2], label='|u_pred - u_exact|')

plt.tight_layout()
plt.savefig('solution_heatmap.png', dpi=150, bbox_inches='tight')
plt.show()

print("Solution heatmap saved to: solution_heatmap.png")

## Save Model and Results

In [ ]:
# Save trained model
torch.save({
    'model_state_dict': trained_model.state_dict(),
    'history': history,
    'final_error': history['relative_l2_error'][-1],
    'config': {
        'architecture': 'MLP',
        'hidden_dims': [64, 64, 64, 64],
        'activation': 'tanh',
        'n_epochs': 20000,
        'n_interior': 10000,
        'n_boundary': 100,
        'learning_rate': 1e-3,
    }
}, 'poisson_pinn_trained.pt')

print("✅ Model saved to: poisson_pinn_trained.pt")
print("\n📦 Download these files:")
print("  1. poisson_pinn_trained.pt (trained model)")
print("  2. training_history.png (loss curves)")
print("  3. solution_heatmap.png (solution visualization)")
print("\n🎯 Next: Continue with Task 3 locally!")